In [ ]:
library(oro.nifti)
library(ggplot2)
library(car)
library(HH)
library(parallel)
library(data.table)

In [ ]:
curr_dir <- getwd()
setwd('./20190701/conn_3/results/firstlevel/SBC_02')
files <- list.files(pattern = "^BETA_Subject[0-9][0-9][0-9]_Condition001_Source001.nii$")

In [ ]:
conn_mean <- function(f) {
    dmn_mask <- readNIfTI('./dmn2_coreg.nii.gz')
    sub <- readNIfTI(f)
    sub_masked <- sub * dmn_mask
    sub_masked_vec <- as.vector(sub_masked@.Data)
    sub_masked_vec <- na.omit(sub_masked_vec)
    conn <- sub_masked_vec[sub_masked_vec != 0]
    return (data.frame(f,mean(conn)))
}
conns <- mclapply(files, conn_mean, mc)

In [ ]:
setwd('./20190701/conn_3/results/firstlevel/SBC_02')
conns_dmnc <- mclapply(files, conn_mean, mc.cores=5)
conns_dmnc <- rbindlist(conns)
setwd(curr_dir)
head(conns_dmnc) 

In [ ]:
covs <- read.table('./20190701/covs.csv', sep=',', header = TRUE, quote = '"')

In [ ]:
data_dmnc <- data.frame(as.vector(conns_dmnc), covs)
head(data_dmnc)

In [ ]:
write.table(data_dmnc, './data_conndmn.csv') #, row.names = FALSE)

In [ ]:
ggplot(data_dmnc, aes(x=diag, y=conns)) +
    geom_boxplot(alpha = 0.2, width = 0.3, aes(fill=factor(diag))) +
    xlab('Diagnosis') +
    ylab('Connectivity DMN') +
    theme(strip.text.x = element_text(size=9, color="black", face="bold")) +
    guides(color=guide_legend("Diagnosis")) +
    geom_jitter(position=position_jitter(width=0.1), aes(colour=factor(diag)), alpha=1)


In [ ]:
df(0.95,2,68)
ancova1 <- ancova(conns ~ diag + Age_All, data=data_dmnc)
ancova1

In [ ]:
df(0.95,2,69)
anova(aov(conns ~ diag, data=data_dmnc))

In [ ]:
ggplot(data_dmnc, aes(x=g_all, y=conns)) + 
  geom_point(size = 4, aes(shape = nas, color=diag)) +
  geom_smooth(method=aov, se=FALSE, fullrange=TRUE, color='darkblue') + 
  ylab('Connectivity DMN') +
  scale_shape_manual(values = c(16, 17, 18)) + 
  theme(legend.position = "bottom")

In [ ]:
summary(lm(conns ~ g_all, data=data_dmnc))

In [ ]:
summary(lm(conns ~ g_all * Age_All, data=data_dmnc))